# Image classifier to see if a person is smiling

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-07-28 15:23:37.853391: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Lets load in our data set and mix em up a bit
seed_train_validation = 1 # Seed for repeatability

train_datagen = ImageDataGenerator(rescale=1.0/255,
                                  zoom_range=0.2,
                                  shear_range=0.2,
                                  rotation_range=10,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

# Load our training dataset
train_val_ds = keras.utils.image_dataset_from_directory(
    directory ='/Users/tktakaro/Documents/ML_Projects/SmilingOrNot/train',
    image_size = (150,150),
    seed = seed_train_validation,
    color_mode='rgb',
    label_mode = 'categorical',
    batch_size=32,
    shuffle = True)


test_datagen = ImageDataGenerator(rescale=1./255)
# Load our test dataset
test_ds = keras.utils.image_dataset_from_directory(
    directory ='/Users/tktakaro/Documents/ML_Projects/SmilingOrNot/test',
    image_size = (150,150),
    seed = seed_train_validation,
    labels=None,
    batch_size=32,
    color_mode = 'rgb',
    shuffle = True)

Found 1203 files belonging to 2 classes.
Found 12030 files belonging to 1 classes.


## Unfortunately test dataset is unlabeled. Can't use for accuracy estimate

In [3]:
# Split the test dataset in thirds to give us a validation dataset too
batches = tf.data.experimental.cardinality(train_val_ds) # How many files in test dataset?
train_ds = train_val_ds.take((2 * batches) // 3)
val_ds = train_val_ds.skip((2 * batches) // 3)

In [4]:
# First, use ResNet as a base model
base_model = keras.applications.ResNet50(weights='imagenet', include_top = False, input_shape=(150,150,3))
# Freeze layers from base_model to only train new specialized layers for this problem
for layer in base_model.layers:
    layer.trainable = False
        
# Build up layers
x = base_model.output    
x = (keras.layers.Dropout(0.2))(x)    
x = keras.layers.Flatten()(x)
# Two categories: Smiling or Not
predictions = keras.layers.Dense(2, activation = 'softmax')(x)

# Set up and compile model
model = keras.models.Model(inputs = base_model.input , outputs =  predictions)
    
model.compile(optimizer='Nadam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Run the fit
model.fit(train_ds, 
    epochs = 10 , validation_data=val_ds,
    steps_per_epoch = len(train_ds),
    validation_steps=len(val_ds),)

Epoch 1/10
25/25 [==============================] - 35s 1s/step - loss: 1.5850 - accuracy: 0.7875 - val_loss: 0.1842 - val_accuracy: 0.9429
Epoch 2/10
25/25 [==============================] - 33s 1s/step - loss: 0.1157 - accuracy: 0.9638 - val_loss: 0.1106 - val_accuracy: 0.9752
Epoch 3/10
25/25 [==============================] - 33s 1s/step - loss: 0.0343 - accuracy: 0.9837 - val_loss: 0.0766 - val_accuracy: 0.9777
Epoch 4/10
25/25 [==============================] - 34s 1s/step - loss: 0.0305 - accuracy: 0.9912 - val_loss: 0.0787 - val_accuracy: 0.9801
Epoch 5/10
25/25 [==============================] - 34s 1s/step - loss: 0.0175 - accuracy: 0.9950 - val_loss: 0.1062 - val_accuracy: 0.9752
Epoch 6/10
25/25 [==============================] - 33s 1s/step - loss: 0.0109 - accuracy: 0.9950 - val_loss: 0.0785 - val_accuracy: 0.9826
Epoch 7/10
25/25 [==============================] - 34s 1s/step - loss: 0.0208 - accuracy: 0.9937 - val_loss: 0.2487 - val_accuracy: 0.9479
Epoch 8/10
25/25 [==

### Further optimization beyond 98% validation accuracy is unnecessary, especially given small dataset